**Anális Exploratorio de datos de Forbes Global 2021**


Fuente: https://www.forbes.com/

Realizado por: Sibyl González

Email: sibylgonzalez@gmail.com


En este notebook se utiliza la API de Forbes Global 2000 para el 2021 y compararlo con el año 2023.La lista Forbes Global 2000 es un ranking anual de 2000 empresas privadas en el mundo por la revista Forbes. La clasificación se basa en una combinación de cuatro indicadores: ventas, utilidades, activos y valor de mercado.

***Importar librerias***

In [8]:
import pandas as pd
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px


***Extracción de datos***

In [9]:
#Construyo una función que extraiga los datos del 2021 y del 2023

def obtener_datos_forbes_global2000(year):
    # Construir la URL del API según el año proporcionado
    api_url = f"https://www.forbes.com/forbesapi/org/global2000/{year}/position/true.json?limit=2000"

    # Definir los encabezados y las cookies
    headers = {
        "accept": "application/json, text/plain, */*",
        "referer": "https://www.forbes.com/global2000/",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36",
    }

    cookies = {
        "notice_behavior": "expressed,eu",
        "notice_gdpr_prefs": "0,1,2:1a8b5228dd7ff0717196863a5d28ce6c",
    }

    # Realizar la solicitud HTTP y obtener la respuesta en formato JSON
    response = requests.get(api_url, headers=headers, cookies=cookies).json()

    # Crear una lista de listas (tabla) con los datos relevantes de la respuesta JSON
    tabla = [
        [
            item["organizationName"],
            item["country"],
            item["revenue"],
            item["profits"],
            item.get("assets", None),
            item["marketValue"]
        ] for item in
        sorted(response["organizationList"]["organizationsLists"], key=lambda k: k["position"])
    ]

    # Crear un DataFrame de pandas con la tabla
    df = pd.DataFrame(tabla, columns=["empresa", "pais", "ventas", "beneficios", "activos", "valor de mercado"])

    return df

# Obtener el DataFrame para el año 2021
df_2021 = obtener_datos_forbes_global2000(2021)
df_2023 = obtener_datos_forbes_global2000(2023)


In [10]:
df_2021

,empresa,pais,ventas,beneficios,activos,valor de mercado
0,ICBC,China,190467.0,45768.7,4914670.0,249532.0
1,JPMorgan Chase,United States,136187.0,40371.0,3689336.0,464822.0
2,Berkshire Hathaway,United States,245510.0,42521.0,873729.0,624408.0
3,China Construction Bank,China,173478.0,39299.2,4301700.0,210435.0
4,Saudi Arabian Oil Company (Saudi Aramco),Saudi Arabia,229747.0,49282.8,510252.0,1897165.0
...,...,...,...,...,...,...
1995,JTEKT,Japan,13050.0,-34.9,11996.0,3467.0
1996,Insurance Australia Group,Australia,5407.0,-212.0,24971.0,9269.0
1997,Sumec Corporation,China,13013.0,79.2,7075.0,1346.0
1998,Gold Fields,South Africa,3888.0,718.1,7473.0,8864.0


In [11]:
df_2023


,empresa,pais,ventas,beneficios,activos,valor de mercado
0,JPMorgan Chase,United States,179933.0,41795.0,3744305.0,399594.0
1,Saudi Arabian Oil Company (Saudi Aramco),Saudi Arabia,589467.0,156357.0,660990.0,2055222.0
2,ICBC,China,216767.0,52470.8,6116824.0,203012.0
3,China Construction Bank,China,203083.0,48252.5,4977476.0,172992.0
4,Agricultural Bank of China,China,186144.0,37924.7,5356862.0,141822.0
...,...,...,...,...,...,...
1995,Alfa Laval,Sweden,5353.0,489.5,7822.0,15595.0
1996,Gap,United States,15616.0,-202.0,11386.0,3175.0
1997,Yes Bank,India,3338.0,91.6,43221.0,5603.0
1998,BEKB-BCBE,Switzerland,556.0,167.1,42972.0,2490.0


***Procesamiento de los datos***

In [12]:
df_2021 = df_2021.sort_values(by='valor de mercado', ascending=False)

df_2023 = df_2023.sort_values(by='valor de mercado', ascending=False)

In [13]:
# Definir una función para obtener empresas viejas comparando DataFrames de diferentes años
def viejas():
    # Obtener la lista de empresas del DataFrame del año 2021
    empresas_2021 = df_2021['empresa'].tolist()
    
    # Obtener la lista de empresas del DataFrame del año 2023
    empresas_actual = df_2023['empresa'].tolist()
    
    # Calcular las empresas únicas presentes en el DataFrame del año 2021 pero no en el del año 2023
    empresas_unicas_2021 = list(set(empresas_2021) - set(empresas_actual))
    
    # Filtrar el DataFrame del año 2021 para obtener solo las empresas viejas
    df_empresas_viejas = df_2021[df_2021['empresa'].isin(empresas_unicas_2021)]
    
    # Devolver el DataFrame resultante
    return df_empresas_viejas

# Definir una función para obtener empresas nuevas comparando DataFrames de diferentes años
def nuevas():
    # Obtener la lista de empresas del DataFrame del año 2021
    empresas_2021 = df_2021['empresa'].tolist()
    
    # Obtener la lista de empresas del DataFrame del año 2023
    empresas_actual = df_2023['empresa'].tolist()
    
    # Calcular las empresas únicas presentes en el DataFrame del año 2023 pero no en el del año 2021
    empresas_unicas_actual = list(set(empresas_actual) - set(empresas_2021))
    
    # Filtrar el DataFrame del año 2023 para obtener solo las empresas nuevas
    df_empresas_nuevas = df_2023[df_2023['empresa'].isin(empresas_unicas_actual)]
    
    # Devolver el DataFrame resultante
    return df_empresas_nuevas


In [12]:
# Obtener DataFrame de empresas viejas utilizando la función 'viejas'
df_dropped = viejas()

# Obtener DataFrame de empresas nuevas utilizando la función 'nuevas'
df_new = nuevas()


In [17]:
# Restablecer el índice del DataFrame de empresas viejas, descartando el índice anterior
df_empresas_viejas = df_empresas_viejas.reset_index(drop=True)

# Restablecer el índice del DataFrame de empresas nuevas, descartando el índice anterior
df_empresas_nuevas = df_empresas_nuevas.reset_index(drop=True)


In [18]:
df_empresas_nuevas

,empresa,pais,ventas,beneficios,activos,valor de mercado
0,Meta Platforms,United States,117346.0,21443.0,184491.0,599818.0
1,The Home Depot,United States,157403.0,17105.0,76445.0,293953.0
2,Shell,United Kingdom,365892.0,43513.1,429154.0,205451.0
3,Salesforce,United States,31352.0,208.0,102583.0,197590.0
4,TotalEnergies,France,257588.0,21120.7,293032.0,151640.0
...,...,...,...,...,...,...
368,Schweizerische Nationalbank,Switzerland,-74515.0,-76169.8,1044932.0,531.0
369,PT Garuda Indonesia (Persero),Indonesia,2110.0,3669.0,6235.0,393.0
370,Coinshares International,United Kingdom,2536.0,2717.9,1660.0,204.0
371,Pakistan State Oil,Pakistan,15779.0,149.5,4160.0,188.0


In [19]:
df_empresas_viejas

,empresa,pais,ventas,beneficios,activos,valor de mercado
0,Facebook,United States,85966.0,29145.0,159316.0,870476.0
1,Home Depot,United States,132110.0,12866.0,70581.0,352758.0
2,Salesforce.com,United States,21252.0,4072.0,66301.0,213589.0
3,Royal Dutch Shell,Netherlands,170189.0,-22166.3,379268.0,151972.0
4,Total,France,119715.0,-7579.0,266132.0,118449.0
...,...,...,...,...,...,...
368,Hyakujushi Bank,Japan,612.0,37.9,51288.0,427.0
369,First Heartland Jusan Bank,Kazakhstan,533.0,1063.0,3837.0,323.0
370,Piraeus Bank,Greece,2652.0,-753.5,87577.0,289.0
371,RMB Holdings,South Africa,2.0,2032.6,330.0,191.0


***Visualizaciones***

In [21]:
# Concatenar los DataFrames df_2021 y df_2023 para crear un DataFrame combinado
combined_df = pd.concat([df_2021, df_2023])

# Crear un gráfico de dispersión con Plotly Express (px)
fig = px.scatter(
    combined_df, x='ventas', y='beneficios', 
    # text='empresa',  # Puedes incluir etiquetas de texto, pero está comentado
    title='Comparación de Ventas vs. Beneficios',
    labels={'ventas': 'Ventas', 'beneficios': 'Beneficios'},
    template='plotly',  # Utilizar el estilo de diseño 'plotly'
    hover_data=["empresa"]  # Mostrar información adicional al pasar el mouse sobre los puntos
)

# Mostrar el gráfico
fig.show()


In [23]:
# Concatenar los DataFrames df_2021 y df_2023 para crear un DataFrame combinado
combined_df = pd.concat([df_2021, df_2023])

# Crear un gráfico de caja (box plot) con Plotly Express (px)
fig = px.box(
    combined_df, x='activos', title='Distribución de Activos',
    labels={'activos': 'Activos'}, template='plotly'
)

fig.update_traces(
    boxpoints='all',
    jitter=0.3, 
    pointpos=-1.8,  
    marker=dict(size=4),
    selector=dict(mode='markers+text', textfont_size=10), 
    textposition='top center'  
)

# Mostrar el gráfico
fig.show()


In [25]:

# Aplicar el logaritmo natural a la columna 'activos' y crear una nueva columna 'log_activos'
combined_df["log_activos"] = combined_df["activos"].apply(lambda x: np.log(x))

# Crear un gráfico de caja (box plot) con Plotly Express (px) utilizando los logaritmos de 'activos'
fig = px.box(
    combined_df, x='log_activos', title='Distribución de Activos (Logaritmo Natural)',
    labels={'activos': 'Activos'}, template='plotly'
)

# Configuraciones adicionales para mejorar la visualización
fig.update_traces(
    boxpoints='log_activos',  # Mostrar todos los puntos para cada caja usando los logaritmos
    jitter=0.3,  
    pointpos=-1.8,  
    marker=dict(size=4), 
    selector=dict(mode='markers+text', textfont_size=10), 
    textposition='top center'  )

# Mostrar el gráfico
fig.show()


In [28]:
# Crear un gráfico de pastel para la distribución de empresas por país en 2021
fig_2021 = px.pie(
    df_2021.head(10), names='pais', title='Distribución de Empresas por País en 2021', template='plotly'
)

fig_2021.update_traces(textinfo='percent+label')

# Mostrar el gráfico de pastel para 2021
fig_2021.show()

# Crear un gráfico de pastel para la distribución de empresas por país actualmente
fig_actual = px.pie(
    df_2021.head(10), names='pais', title='Distribución de Empresas por País Actualmente', template='plotly'
)

fig_actual.update_traces(textinfo='percent+label')

# Mostrar el gráfico de pastel actual
fig_actual.show()




In [37]:

# Crear un gráfico de barras para los países que perdieron más posiciones en el top
fig_2021_bar = px.bar(
    df_empresas_viejas, 
    y='pais',  # La columna que representa los países
    orientation='h',  # Barra horizontal
    title='Países que Perdieron Más Posiciones en el Top',
    labels={'posiciones_perdidas': 'Posiciones Perdidas', 'pais': 'País'},
    template='plotly'
)

# Mostrar el gráfico de barras para posiciones perdidas
fig_2021_bar.show()


In [39]:

# Crear un gráfico de barras horizontales para la distribución de empresas por país en la actualidad
fig_actual_bar = px.bar(
    df_2023, 
    y='pais',  # La columna que representa los países
    orientation='h',  # Barra horizontal
    title='Países con Más Empresas Nuevas en el Top',
    labels={'cantidad_empresas': 'Cantidad de Empresas', 'pais': 'País'},
    template='plotly'
)

# Mostrar el gráfico de barras horizontales para empresas nuevas
fig_actual_bar.show()


***Conclusiones***


En la primera gráfica, se observa que las empresas del sector tecnológico, como Apple y Microsoft, se destacan por sus altos beneficios. Esto se debe a que estas empresas operan en mercados globales y tienen una gran cuota de mercado. Por otro lado, empresas como Walmart, que se centran en el comercio minorista, tienen altos niveles de ventas, pero sus beneficios son menores. Esto se debe a que el comercio minorista es un mercado más competitivo y las empresas tienen que invertir más en infraestructura y marketing.

La segunda gráfica muestra que un pequeño número de empresas concentran un gran volumen de activos. Esto se debe a que estas empresas son las más grandes y tienen una presencia global.

La tercera gráfica muestra que alrededor del 75% de las empresas de la lista Forbes Global 2001 y 2023 tienen sede en 11 países. Estados Unidos y China son los países con más empresas, seguidos de Japón, Reino Unido y Corea del Sur.

La cuarta gráfica muestra un aumento en el número de empresas estadounidenses en la lista. En 2023, Estados Unidos tenía 611 empresas en la lista, lo que representa un aumento del 2,5% respecto a 2022. China, por otro lado, perdió 15 empresas en la lista, lo que representa una disminución del 2,2%.

En general, los gráficos de Forbes Global 2000 muestran que las empresas tecnológicas, las empresas con sede en Estados Unidos y las empresas de los sectores energéticos y de consumo son las más grandes y rentables del mundo.